In [11]:
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch



In [13]:
# Загрузим набор данных с эмоциями
dataset = load_dataset("emotion")
accuracy_metric = load_metric("accuracy")

# Разделим данные на тренировочные и тестовые
train_data = dataset['train']
test_data = dataset['test']


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)  # Get the class with the highest probability
    return accuracy_metric.compute(predictions=predictions, references=labels)


C:\Users\Assylzhan\miniconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [3]:
print(dataset)
print(dataset['train'][0])


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
{'text': 'i didnt feel humiliated', 'label': 0}


In [21]:
# Загрузим токенизатор для BERT
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Функция для токенизации данных
def preprocess_data(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=65)




# Применим токенизацию к данным
train_data = train_data.map(preprocess_data, batched=True)
test_data = test_data.map(preprocess_data, batched=True)



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

C:\Users\Assylzhan\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Assylzhan\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [5]:
print(train_data[0])


{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [101, 1045, 2134, 2102, 2514, 26608, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [22]:
# Загрузим предобученную модель BERT для классификации
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=6)  # 6 — количество эмоций в наборе данных
model.config.max_position_embeddings = 65
print(train_data[0]['input_ids'])  # Убедитесь, что длина совпадает с `max_length`



model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[0, 118, 46405, 619, 32386, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [23]:
# Определим параметры обучения
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # Здесь обновлено
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)


# Создадим объект Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    compute_metrics=compute_metrics,  # Add this to calculate accuracy
)



In [24]:
# Запустим обучение
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.267200,0.200913,0.924500
2,0.165700,0.180016,0.930000
3,0.100000,0.157657,0.930000


TrainOutput(global_step=3000, training_loss=0.2519659563700358, metrics={'train_runtime': 6195.6215, 'train_samples_per_second': 7.747, 'train_steps_per_second': 0.484, 'total_flos': 1603390576320000.0, 'train_loss': 0.2519659563700358, 'epoch': 3.0})

In [25]:
# Оценка на тестовых данных
results = trainer.evaluate()
print(results)
    

{'eval_loss': 0.15765661001205444, 'eval_accuracy': 0.93, 'eval_runtime': 60.9366, 'eval_samples_per_second': 32.821, 'eval_steps_per_second': 4.103, 'epoch': 3.0}


In [18]:
# Сохраним модель
model.save_pretrained("emotion_model")
tokenizer.save_pretrained("emotion_model")


('emotion_model\\tokenizer_config.json',
 'emotion_model\\special_tokens_map.json',
 'emotion_model\\vocab.txt',
 'emotion_model\\added_tokens.json',
 'emotion_model\\tokenizer.json')

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

# Load the model with the updated configuration
model = AutoModelForSequenceClassification.from_pretrained("emotion_model", ignore_mismatched_sizes=True)
tokenizer = AutoTokenizer.from_pretrained("emotion_model")

# Create a pipeline using the loaded model and tokenizer
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Test the model with a new text
text = "I feel so happy today!"
result = classifier(text)
print(result)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at emotion_model and are newly initialized because the shapes did not match:
- bert.embeddings.position_embeddings.weight: found shape torch.Size([512, 768]) in the checkpoint and torch.Size([65, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.24399344623088837}]
